In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('../input/hotel-booking/hotel_booking.csv')

In [ ]:
df.head()

**Cleaning the data**

In [ ]:
df.isna().sum()

In [ ]:
def data_clean(df):
    df.fillna(0 , inplace =True)

In [ ]:
data_clean(df)

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
list =['adults', 'children', 'babies']
for i in list :
    print('{} has uniquevalues as {}' .format(i,df[i].unique()))

In [ ]:
pd.set_option('display.max_columns',32)

In [ ]:
filter =(df['children']==0) & (df['adults'] ==0) & (df['babies']==0)
df[filter]

In [ ]:
data = df[~filter]

**from which country vistors come ?**

In [ ]:
data.columns

In [ ]:
country_wise_data=data[data['is_canceled'] ==0]['country'].value_counts().reset_index()

In [ ]:
country_wise_data.columns=['country' ,'No of guests']

In [ ]:
country_wise_data

In [ ]:
!pip install folium

In [ ]:
import folium 
from folium.plugins import HeatMap

In [ ]:
basemap =folium.Map()

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
map_guest= px.choropleth(country_wise_data,
                        locations =country_wise_data['country'],
                        color =country_wise_data['No of guests'],
                        hover_name =country_wise_data['country'],
                        title ="Home country of guests")

In [ ]:
map_guest.show()

**How much guests pay per night**

In [ ]:
data2= data[data['is_canceled']==0]

In [ ]:
data2.columns

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='reserved_room_type',y='adr',hue ='hotel',data=data2)
plt.title('Price of room types per night & per person')
plt.xlabel('Room type')
plt.ylabel('Price(EUR)')
plt.legend()
plt.show()

In [ ]:
data['reserved_room_type']

**price per night over the year**

In [ ]:
data.columns

In [ ]:
data['hotel']

In [ ]:
data_resort = data[(data['hotel']=='Resort Hotel') & (data['is_canceled']==0)]
data_resort

In [ ]:
data_city = data[(data['hotel']=='City Hotel') & (data['is_canceled']==0)]
data_city

In [ ]:
resort_hotel=data_resort.groupby(['arrival_date_month'])['adr'].mean().reset_index()

In [ ]:
city_hotel=data_city.groupby(['arrival_date_month'])['adr'].mean().reset_index()

In [ ]:
final =resort_hotel.merge(city_hotel, on='arrival_date_month')

In [ ]:
final.columns =['months' ,'price for_resort' ,'price for_city']

In [ ]:
final

In [ ]:
!pip install sorted-months-weekdays

In [ ]:
!pip install sort-dataframeby-monthorweek

In [ ]:
import sort_dataframeby_monthorweek as sd


In [ ]:
def sort_data(df,colname):
    return sd.Sort_Dataframeby_Month(df,colname)

In [ ]:
final =sort_data(final ,'months')

In [ ]:
px.line(final, x='months' ,y=['price for_resort' ,'price for_city'] ,title='room Price per month' )


**rush of guests and when hotels hame may guests**

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
rush_resort =data_resort['arrival_date_month'].value_counts().reset_index()
rush_resort.columns = ['months ','rush for resort']
rush_resort

In [ ]:
rush_city =data_city['arrival_date_month'].value_counts().reset_index()
rush_city.columns = ['months ','rush for city']
rush_city

In [ ]:
final_rush = rush_resort.merge(rush_city)
final_rush

In [ ]:
final_rush.columns =['month' ,'No guest for resort','No guest for city']
final_rush

In [ ]:
final_rush =sort_data(final_rush ,'month')

In [ ]:
px.line(final_rush ,x='month' ,y =['No guest for resort', 'No guest for city'] ,title='no of guests each year')

In [ ]:
data.corr()

In [ ]:
co_relation= data.corr()['is_canceled'].sort_values(ascending=False)

In [ ]:
co_relation

In [ ]:
data.groupby('is_canceled')['reservation_status'].value_counts()

In [ ]:
list_not = ['days_in_waiting_list','arrival_date_year']

In [ ]:
num_features=[col for col in data.columns if data[col].dtype!='O' and col not in list_not]

In [ ]:
data.columns

In [ ]:
cot_not =['arrival_date_year','booking_changes','assigned_room_type','reservation_status','days_in_waiting_list','country']

In [ ]:
cot_not

In [ ]:
cat_features=[col for col in data.columns if data[col].dtype=='O' and col not in cot_not]

In [ ]:
cat_features

In [ ]:
cat_features

In [ ]:
data_cat=data[cat_features]

In [ ]:
data_cat.head()

In [ ]:
import warnings
from  warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
data_cat['reservation_status_date'] =pd.to_datetime(data_cat['reservation_status_date'])

In [ ]:
data_cat['year'] =data_cat['reservation_status_date'].dt.year
data_cat['month'] =data_cat['reservation_status_date'].dt.month
data_cat['day'] =data_cat['reservation_status_date'].dt.day

In [ ]:
data_cat.drop('reservation_status_date',axis=1,inplace=True)

In [ ]:
data_cat

In [ ]:
data_cat['cancellation'] =data['is_canceled']

In [ ]:
data_cat

In [ ]:
data_cat.columns

In [ ]:
cols =data_cat.columns[0:8]

In [ ]:
cols

In [ ]:
for col in cols:
    dict = data_cat.groupby([col])['cancellation'].mean().to_dict()
    data_cat[col] =data_cat[col].map(dict)

In [ ]:
data_cat.head()

In [ ]:
data_cat.drop(['email','phone-number','credit_card'],axis=1  , inplace=True)

In [ ]:
data_cat.drop(['name'],axis=1  , inplace=True)

In [ ]:
datafram= pd.concat([data_cat, data[num_features]],axis=1)

In [ ]:
datafram.drop('cancellation',axis=1,inplace=True)

In [ ]:
datafram.info()

In [ ]:
sns.distplot(datafram['lead_time'])

In [ ]:
def handel_outlier(col):
    datafram[col] =np.log1p(datafram[col])

In [ ]:
handel_outlier('lead_time')

In [ ]:
sns.distplot(datafram['lead_time'])

In [ ]:
sns.distplot(datafram['adr'])

In [ ]:
handel_outlier('adr')

In [ ]:
sns.distplot(datafram['adr'])

In [ ]:
datafro_col = [datafram.columns]

In [ ]:
datafro_col

In [ ]:
# for col in datafro_col :
#     sns.distplot(datafram[col])    
#     plt.show()

In [ ]:
datafram.isna().sum()

In [ ]:
datafram.dropna(inplace=True)

In [ ]:
y = datafram['is_canceled']

In [ ]:
x = datafram.drop('is_canceled',axis=1)

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

In [ ]:
feature_sel_model=SelectFromModel(Lasso(alpha=0.005 , random_state=42))

In [ ]:
feature_sel_model.fit(x,y)

In [ ]:
feature_sel_model.get_support()

In [ ]:
cols= x.columns

In [ ]:
selected_feature = cols[feature_sel_model.get_support()]

In [ ]:
print('total_features {}' .format(x.shape[1]))
print('total_features {}' .format(len(selected_feature)))

In [ ]:
selected_feature

In [ ]:
x = x[selected_feature]
x.info()

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train ,y_test = train_test_split(x , y , test_size=.3 , random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg =LogisticRegression(solver='liblinear')

In [ ]:
logreg.fit(x,y)

In [ ]:
y_predict =logreg.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix ,accuracy_score


In [ ]:
confusion_matrix(y_predict,y_test)

In [ ]:
accuracy_score(y_test , y_predict)

In [ ]:
from sklearn.model_selection import cross_val_score


In [ ]:
score=cross_val_score(logreg,x,y,cv=10)

In [ ]:
score.mean()

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
models = []
models.append(('LogisticRegression',LogisticRegression(solver='liblinear')))
models.append(('Naive bayes' ,GaussianNB()))
models.append(('RandomForest' ,RandomForestClassifier()))
models.append(('Decision Tree ' ,DecisionTreeClassifier()))
models.append (('KNN' ,KNeighborsClassifier()))


In [ ]:
for name , model in models :
    print(name)
    model.fit(x_train , y_train)
    
    prediction = model.predict(x_test)
    
    from sklearn.metrics import confusion_matrix
    
    print(confusion_matrix(prediction,y_test))
    print('\n')
    print(accuracy_score(prediction,y_test))
    print('\n')